# Train

```{note}
现在有了 model, tokenizer 和 dataset, 我们可以开始训练了。
```

## Configuration

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import os
from tqdm import tqdm

# Imports from local files
from torch_train.torch_tokenizer import BPETokenizer
from torch_train.torch_model import TransformerModel
from torch_train.torch_dataset import PretrainDataset

# ==========================================
# Configuration
# ==========================================
CONFIG = {
    # Data
    "parquet_path": "data/wikitext-103-raw-v1-train-sampled.parquet", # Use sample set for lower resource usage
    "tokenizer_path": "wiki-tokenizer-1.json",
    "seq_len": 512,           # Reduced context window
    "batch_size": 8,         # Reduced batch size
    
    # Model (Tiny size for stability)
    "d_model": 256,          # Reduced d_model
    "n_head": 8,
    "d_hidden": 1024,         # Reduced d_hidden
    "n_layer": 16,            # Reduced n_layer
    "dropout": 0.1,
    
    # Training
    "lr": 3e-4,
    "epochs": 2,             # Increased epochs since data is smaller
    "log_interval": 10,
    "save_path": "llm_checkpoint_27m.pt",
    "device": "cuda" if torch.cuda.is_available() else "cpu"
}

模型的参数量：
1. 除 transformer block 外的参数数量：
    *  embedding 层：`d_model * vocab_size`
    *  output 层：`vocab_size * d_model`
    *  RMSNorm：`d_model`
2.  每个 transformer block 的参数数量：
    *  attention 的参数量：
        *  qkv 投影：`3 * d_model * d_model`
        *  o 投影：`d_model * d_model`
        *  总的和 QK 的 RMSNorm：`3 * d_model`
    *  feed forward network 的参数量：
        *  gate 和 up projection：`2 * d_model * d_hidden`
        *  down projection：`d_hidden * d_model`


In [2]:
d_model = CONFIG["d_model"]
vocab_size = 20000
n_layer = CONFIG["n_layer"]
d_hidden = CONFIG["d_hidden"]
d_model * (2 * vocab_size + 1 ) + n_layer * (4 * d_model * d_model + 3 * d_model * d_hidden + 4 * d_model)


27033856

## 加载 tokenizer

In [12]:
print(f"Using device: {CONFIG['device']}")

print("Loading Tokenizer...")
tokenizer = BPETokenizer()
tokenizer.load(CONFIG["tokenizer_path"])
print(f"Tokenizer loaded. Vocab size: {len(tokenizer.vocab)}")

vocab_size = len(tokenizer.vocab)
pad_id = tokenizer.special_tokens.get("<PAD>", 0)

Using device: cuda
Loading Tokenizer...
Tokenizer loaded. Vocab size: 20000


## DataLoader

In [4]:
print(f"Loading data from {CONFIG['parquet_path']}...")
file_paths = [CONFIG["parquet_path"]] 

print("Initializing Dataset...")
# Request seq_len + 1 to handle input/target shift
dataset = PretrainDataset(file_paths, tokenizer, seq_len=CONFIG["seq_len"] + 1)

Loading data from data/wikitext-103-raw-v1-train-sampled.parquet...
Initializing Dataset...
Processing 1 files...


Tokenizing texts: 100%|██████████| 2944/2944 [01:01<00:00, 48.00it/s]


Total tokens: 12880663. Total samples (seq_len=513): 25109


In [5]:
dataloader = DataLoader(
    dataset, 
    batch_size=CONFIG["batch_size"], 
    shuffle=True, 
    num_workers=2
)

## 模型

In [6]:
print("Initializing Model...")
model = TransformerModel(
    d_model=CONFIG["d_model"],
    n_head=CONFIG["n_head"],
    d_hidden=CONFIG["d_hidden"],
    n_layer=CONFIG["n_layer"],
    vocab_size=vocab_size,
    max_seq_len=CONFIG["seq_len"] + 1, 
    dropout=CONFIG["dropout"]
).to(CONFIG["device"])

print(f"Model parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.2f}M")

Initializing Model...
Model parameters: 27.03M


## Training Loop

In [7]:
from torch.optim.lr_scheduler import CosineAnnealingLR

optimizer = optim.AdamW(model.parameters(), lr=CONFIG["lr"])
criterion = nn.CrossEntropyLoss(ignore_index=pad_id)

# Total steps for scheduler
total_steps = len(dataloader) * CONFIG["epochs"]
scheduler = CosineAnnealingLR(optimizer, T_max=total_steps)

model.train()

print("Starting training...")
for epoch in range(CONFIG["epochs"]):
    total_loss = 0
    # 使用 tqdm 包装 dataloader
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{CONFIG['epochs']}", unit="batch")
    
    for batch_idx, data in enumerate(pbar):
        data = data.to(CONFIG["device"])
        
        # Input: [B, seq_len], Target: [B, seq_len] (shifted)
        input_ids = data[:, :-1]
        target_ids = data[:, 1:]

        # Generate position_ids
        B, T = input_ids.shape
        position_ids = torch.arange(T, device=CONFIG["device"]).unsqueeze(0).expand(B, T)
        
        optimizer.zero_grad()
        output = model(input_ids, position_ids)
        
        loss = criterion(output.reshape(-1, vocab_size), target_ids.reshape(-1))
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        
        # 更新进度条上的 loss 和 LR
        current_lr = scheduler.get_last_lr()[0]
        pbar.set_postfix({"loss": f"{loss.item():.4f}", "lr": f"{current_lr:.2e}"})
        
        # 每 200 step 打印一次 loss
        if (batch_idx + 1) % 200 == 0:
             print(f"Epoch {epoch+1} | Step {batch_idx+1}/{len(dataloader)} | Loss: {loss.item():.4f} | LR: {current_lr:.2e}")
            
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1} Complete. Average Loss: {avg_loss:.4f}")
    
    torch.save(model.state_dict(), CONFIG["save_path"])
    print(f"Checkpoint saved to {CONFIG['save_path']}")

Starting training...


Epoch 1/2:   6%|▋         | 200/3139 [00:50<12:25,  3.94batch/s, loss=6.3364, lr=2.99e-04]

Epoch 1 | Step 200/3139 | Loss: 6.3364 | LR: 2.99e-04


Epoch 1/2:  13%|█▎        | 400/3139 [01:40<11:23,  4.01batch/s, loss=5.9198, lr=2.97e-04]

Epoch 1 | Step 400/3139 | Loss: 5.9198 | LR: 2.97e-04


...

Epoch 2 | Step 2800/3139 | Loss: 4.2579 | LR: 2.15e-06


Epoch 2/2:  96%|█████████▌| 3000/3139 [12:39<00:34,  4.02batch/s, loss=4.3050, lr=3.63e-07]

Epoch 2 | Step 3000/3139 | Loss: 4.3050 | LR: 3.63e-07


Epoch 2/2: 100%|██████████| 3139/3139 [13:14<00:00,  3.95batch/s, loss=4.3537, lr=0.00e+00]


Epoch 2 Complete. Average Loss: 4.3229
Checkpoint saved to llm_checkpoint_27m.pt


## 推理

In [10]:
import math

# 在测试集上进行评估
test_path = "data/wikitext-103-raw-v1-test.parquet"
print(f"Loading test data from {test_path}...")
test_file_paths = [test_path]
test_dataset = PretrainDataset(test_file_paths, tokenizer, seq_len=CONFIG["seq_len"] + 1)
test_dataloader = DataLoader(test_dataset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=2)

def calculate_perplexity(model, dataloader, device):
    model.eval()
    total_loss = 0
    total_steps = 0
    criterion = nn.CrossEntropyLoss(ignore_index=pad_id)
    
    print("Calculating Perplexity...")
    with torch.no_grad():
        for data in tqdm(dataloader, desc="Evaluating"):
            data = data.to(device)
            input_ids = data[:, :-1]
            target_ids = data[:, 1:]
            
            B, T = input_ids.shape
            position_ids = torch.arange(T, device=device).unsqueeze(0).expand(B, T)
            
            output = model(input_ids, position_ids)
            loss = criterion(output.reshape(-1, vocab_size), target_ids.reshape(-1))
            
            total_loss += loss.item()
            total_steps += 1
            
    avg_loss = total_loss / total_steps
    ppl = math.exp(avg_loss)
    return avg_loss, ppl

test_loss, test_ppl = calculate_perplexity(model, test_dataloader, CONFIG["device"])
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Perplexity: {test_ppl:.4f}")

Loading test data from data/wikitext-103-raw-v1-test.parquet...
Processing 1 files...


Tokenizing texts: 100%|██████████| 62/62 [00:01<00:00, 40.49it/s]


Total tokens: 304473. Total samples (seq_len=513): 594
Calculating Perplexity...


Evaluating: 100%|██████████| 75/75 [00:06<00:00, 11.99it/s]

Test Loss: 4.4311
Test Perplexity: 84.0240


In [11]:
def generate(model, tokenizer, prompt, max_new_tokens=50):
    model.eval()
    
    # 1. Tokenize the prompt
    input_ids = tokenizer.encode(prompt)
    input_tensor = torch.tensor(input_ids, dtype=torch.long, device=CONFIG["device"]).unsqueeze(0)
    
    print(f"Prompt: {prompt}")
    print("-" * 40)
    
    generated = input_ids.copy()
    
    with torch.no_grad():
        for _ in range(max_new_tokens):
            # 2. Crop context if too long (keep only the last seq_len tokens)
            ctx = input_tensor[:, -CONFIG["seq_len"]:]
            
            # 3. Create position IDs
            B, T = ctx.shape
            position_ids = torch.arange(T, device=CONFIG["device"]).unsqueeze(0).expand(B, T)
            
            # 4. Forward pass
            output = model(ctx, position_ids)
            
            # 5. Get the logits for the last token
            next_token_logits = output[0, -1, :]
            
            # 6. Greedy decoding: choose the token with the highest probability
            next_token_id = torch.argmax(next_token_logits).item()
            
            # 7. Append the predicted token
            generated.append(next_token_id)
            input_tensor = torch.cat((input_tensor, torch.tensor([[next_token_id]], device=CONFIG["device"])), dim=1)
            
            # 8. Stop if <EOS> is generated
            if next_token_id == tokenizer.special_tokens.get("<EOS>"):
                break
                
    # 9. Decode the generated tokens to string
    decoded = tokenizer.decode(generated)
    print(decoded)
    print("=" * 40)
    return decoded

# Sample prompts
prompts = [
    "The weather is",
    "It was a",
    "In the early",
    "The game began with"
]

for p in prompts:
    generate(model, tokenizer, p)

Prompt: The weather is
----------------------------------------
The weather is a tropical cyclone that is a tropical cyclone , and is a tropical cyclone that is a tropical storm . The storm is a tropical storm , and is a tropical storm . The storm is a tropical storm , and the storm is a tropical storm . The storm
Prompt: It was a
----------------------------------------
It was a member of the Royal Navy . The ship was a ship of the ship 's main armament , and the ship was built in the Atlantic Ocean . The ship was built in the Pacific Ocean , and the ship was built in the Atlantic Ocean . The ship
Prompt: In the early
----------------------------------------
In the early 20th century . The first of the first two @-@ third @-@ century @-@ century @-@ style castles were the first of the first two @-@ third @-@ century American , the first of the first two @-@ third @-@ century American , the first of the first two
Prompt: The game began with
----------------------------------------
The 